In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input , Dense , Conv2D , Flatten , MaxPool2D
from tensorflow.keras import Model
from tensorflow.nn import softmax , relu
from tensorflow.keras.utils import to_categorical

print("Verion of tensorflow is : ",tf.__version__)

Verion of tensorflow is :  2.3.0


In [16]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

In [17]:
print("Shape of x_train : ",x_train.shape)
print("Shape of y_train : ",y_train.shape)
print("Shape of x_test : ",x_test.shape)
print("Shape of y_test : ",y_test.shape)

Shape of x_train :  (60000, 28, 28, 1)
Shape of y_train :  (60000, 10)
Shape of x_test :  (10000, 28, 28, 1)
Shape of y_test :  (10000, 10)


In [18]:
class MyOwnModel(Model):
    def __init__(self):
        super(MyOwnModel , self).__init__()
        self.conv = Conv2D(32 , 3 , name="conv1")
        self.pool = MaxPool2D((2,2))
        self.flatten = Flatten()
        self.dens1 = Dense(128 , activation=relu , name="dense1")
        self.dens2 = Dense(10 , activation=softmax , name="output")
    def call(self , x):
        x = self.conv(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dens1(x)
        return self.dens2(x)

model = MyOwnModel()

In [5]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [75]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def optimizer_for_single_batch(model , data , label):
    with tf.GradientTape() as tape:
        pred_label = model(data)
        loss = loss_obj(label , pred_label)
    grad = tape.gradient(loss , model.trainable_variables)
    optimizer.apply_gradients(zip(grad , model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label , pred_label)

@tf.function
def test_loss_func(model , data , label):
    pred = model(data)
    loss = loss_obj(label , pred)

    test_loss(loss)
    test_accuracy(label , pred)

In [77]:
epochs = 5
model = MyOwnModel()
for epoch in range(epochs):
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for (data , label) in train_ds:
        optimizer_for_single_batch(model , data, label)
    for (data , label) in test_ds:
        test_loss_func(model , data , label)
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.1310122013092041, Accuracy: 96.10333251953125, Test Loss: 0.07420618832111359, Test Accuracy: 97.5999984741211
Epoch 2, Loss: 0.04516156017780304, Accuracy: 98.5816650390625, Test Loss: 0.05397367104887962, Test Accuracy: 98.40999603271484
Epoch 3, Loss: 0.026077941060066223, Accuracy: 99.19999694824219, Test Loss: 0.05923439934849739, Test Accuracy: 98.20999908447266
Epoch 4, Loss: 0.018007274717092514, Accuracy: 99.42666625976562, Test Loss: 0.06015488877892494, Test Accuracy: 98.25
Epoch 5, Loss: 0.01268906518816948, Accuracy: 99.56832885742188, Test Loss: 0.06689789146184921, Test Accuracy: 98.18000030517578
